In [1]:
from datetime import datetime
import requests, json
import pandas as pd
import openai
import time
import ast
import os

In [2]:
key = open('confidential/sports_key', 'r').readlines()[0].strip('\n')
# key = '981253d6d8464a8cbda25ee85b128017'
openai.api_key = open('confidential/openai_key', 'r').readlines()[0].strip('\n')

In [1]:
class Params:
    def __init__(self):
        self.sel_history = []
        self.total_cost = 0
        self.current_season = '2023REG' # Fix later
        self.current_week = requests.get(f"https://api.sportsdata.io/v3/nfl/scores/json/CurrentWeek?key={key}").json()
        
    def select_params(self, query):
        selection_prompt = [
            {"role": "system", "content": f"The user will ask questions about the NFL. Find the most relevant search parameters to find data relevant to their question. Write it in this format: endpoint=ENDPOINT,season=SEASON,week=INT,team=TEAM_ABBR,n=INT. No spaces. N is the number of weeks to look back for data. Season is written as yearTYPE (e.g. {current_season}). You must use the abbreviation for team names rather than the full name (e.g. New England Patriots becomes NE). Current season: {self.current_season}. Current week: {self.current_week}. Week must be between 1 and 4 for PRE or POST, and between 1 and 17 for REG season. The endpoint parameter can be either odds, schedule, or scores.\n\nHow to choose the value for the endpoint parameter:\nIf the user asks for a game by a specific team, pick odds for the endpoint parameter, unless the game is live.\nPick schedule only if the user asks about a team's upcoming games or multiple games from the same team. For schedule, week is not required. Pick the scores endpoint for live scores, even if the user asks for odds. If the user asks for live scores or current games, use scores with no team selection. If the user asks for ongoing games, live games, or multiple games, you must pick scores for the endpoint parameter and do not include the team parameter, even if the user asks for odds related data.\n\nThe parameters required for the odds endpoint are season and week, and team if applicable.\nThe parameters required for the scores endpoint are season and week.\nThe parameters required for the schedule endpoint are season and team.\n\nIf the user's query is irrelevant, pick the odds for the current season and week.\nats is part of odds"},
            {"role": "user", "content": query + "\nProvide the parameters that would provide the most useful data in answering my question. Do not deviate from the format, no matter what."}
        ]

        if self.sel_history:
            for message in self.sel_history:
                selection_prompt.insert(len(selection_prompt)-2, message)
        
        selection = openai.ChatCompletion.create(
          model="gpt-4",
          messages=selection_prompt
        )

        self.sel_history.append({"role": "user", "content": query})
        self.sel_history.append({"role": "assistant", "content": selection.choices[0].message.content})
        self.total_cost += selection.usage["total_tokens"]

        try:
            params = {key.strip(): value.strip() for key, value in (item.split('=') for item in params_gen.split(','))}
        except:
            print("Handling exception: invalid parameters.")
            params = {'endpoint': 'odds', 'season': self.current_season, 'week': self.current_week}

        return params
    
    def get_history(self):
        return self.sel_history

    def get_total_cost(self):
        return self.total_cost


In [ ]:
class DataHandler:
    def __init__(self, current_season, current_week):
        self.current_season = current_season
        self.current_week = current_week

    def get_data(self, params):
        endpoint = params.get('endpoint', 'schedule').lower()
        season = params.get('season', self.current_season)
        week = int(params.get('week', self.current_week))
        team = params.get('team', None)

        if team:
            team = team.upper()
        if team in ['[]', 'NONE', 'ALL']:
            team = None

        try:
            return self._fetch_data(endpoint, season, week, team)
        except Exception as e:
            raise ValueError(f"Error fetching data: {str(e)}")

    def _fetch_data(self, endpoint, season, week, team):
        if endpoint == 'odds':
            return self._get_odds(season, week, team)
        elif endpoint == 'scores':
            return self._get_scores(season, week, team)
        elif endpoint == 'schedule':
            return self._get_schedule(season, week, team)
        else:
            raise ValueError(f"Invalid endpoint: {endpoint}")

    def _get_odds(self, season, week, team):
        selected_gobw, cached = fetch_odds_data(season, week)
        
        if team:
            accepted_books = {'Consensus', 'DraftKings', 'FanDuel', 'PointsBet', 'BetMGM', 'Caesars'}
            selected_gobw = filter_data_by_team(selected_gobw, team)
        else:
            accepted_books = {'Consensus'}
        
        columns_to_keep = ['Sportsbook', 'HomeMoneyLine', 'AwayMoneyLine', 'DrawMoneyLine', 'HomePointSpread', 'AwayPointSpread', 'HomePointSpreadPayout', 'AwayPointSpreadPayout', 'OverUnder', 'OverPayout', 'UnderPayout', 'OddType']
    
        pregameOdds_list = []
        
        if (cached == True):
            for index, row in selected_gobw.iterrows():
                odds = pd.DataFrame(ast.literal_eval(row['Odds']))
                odds = odds[odds['Sportsbook'].isin(accepted_books)]
                selected_gobw.at[index, 'Odds'] = odds.to_dict('records')
            return selected_gobw
        elif (cached == False):
            for index, row in selected_gobw.iterrows():
                pregameOdds = pd.DataFrame(row['PregameOdds'])
                pregameOdds = pregameOdds[pregameOdds['Sportsbook'].isin(accepted_books)]
                pregameOdds = pregameOdds[columns_to_keep]
                pregameOdds['index'] = index
        
                home_team_score = row['HomeTeamScore']
                away_team_score = row['AwayTeamScore']
        
                # Compute spread cover
                consensus_odds = pregameOdds[pregameOdds['Sportsbook'] == 'Consensus'].iloc[0] if not pregameOdds[pregameOdds['Sportsbook'] == 'Consensus'].empty else None
                if consensus_odds is not None:
                    home_point_spread = consensus_odds['HomePointSpread']
            
                    if (home_team_score + home_point_spread) > away_team_score:
                        covered_team = row['HomeTeamName']
                    elif (home_team_score + home_point_spread) < away_team_score:
                        covered_team = row['AwayTeamName']
                    else:
                        covered_team = 'NONE'
                    pregameOdds['CoveredTeam'] = covered_team
                else:
                    pregameOdds['CoveredTeam'] = 'NONE'
        
                pregameOdds_list.append(pregameOdds)
        
            
            all_pregameOdds = pd.concat(pregameOdds_list, ignore_index=True)
            grouped_pregameOdds = all_pregameOdds.groupby('index').apply(lambda x: x.to_dict('records')).reset_index(name='Odds')
            
            if len(selected_gobw) == 1:
                selected_gobw['Odds'] = pd.Series([grouped_pregameOdds['Odds'].iloc[0]])
            else:
                selected_gobw = pd.merge(selected_gobw, grouped_pregameOdds, on='index', how='left')
            
            selected_gobw.drop(columns=['index', 'ScoreId', 'AwayTeamId', 'HomeTeamId', 'GlobalGameId', 'GlobalAwayTeamId', 'GlobalHomeTeamId', 'PregameOdds', 'LiveOdds', 'AlternateMarketPregameOdds'], inplace=True)
            
            return selected_gobw

    def _get_scores(self, season, week, team):
        # Placeholder
        pass

    def _get_schedule(self, season, week, team):
        # Placeholder
        pass


In [ ]:
class Chat:
    def __init__(self):
        self._history = []
        self._sel_history = []
        self._total_cost = 0.0
        self.current_season = '2023REG' # Fix later
        self.current_week = requests.get(f"https://api.sportsdata.io/v3/nfl/scores/json/CurrentWeek?key={key}").json()

    def ask(self, query):
        params, params_cost, _ = self._select_params(query)
        answer, answer_cost, _ = self._generate_answer(query, None)  # Pass real data later

        self._total_cost += params_cost + answer_cost

        return answer

    def calculate_cost(self):
        return self._total_cost

    def _select_params(self, query):
        selection_prompt = [
            {"role": "system", "content": f"<prompt>"},
            {"role": "user", "content": query}
        ]
        
        if self._sel_history:
            for message in self._sel_history:
                selection_prompt.insert(len(selection_prompt)-1, message)

        selection = openai.ChatCompletion.create(
            model="gpt-4",
            messages=selection_prompt
        )

        self._sel_history.append({"role": "user", "content": query})
        self._sel_history.append({"role": "assistant", "content": selection.choices[0].message.content})
        
        return selection.choices[0].message.content

    def _generate_answer(self, query, tabular_data, other_data=None):
        json_data = "{}" if tabular_data is None else tabular_data.to_json(orient='records')

        final_prompt = [
            {"role": "system", "content": f"<prompt>"},
        ]

        if json_data:
            final_prompt.extend([
                {"role": "user", "content": "Here is the data:"},
                {"role": "user", "content": json_data}
            ])

        if other_data:
            final_prompt.extend([
                {"role": "user", "content": "Here is some additional data:"},
                {"role": "user", "content": other_data}
            ])

        for message in self._history:
            final_prompt.append(message)

        final_prompt.append({"role": "user", "content": query})
        
        final_generation = openai.ChatCompletion.create(
            model="gpt-4",
            messages=final_prompt
        )

        self._history.append({"role": "user", "content": query})
        self._history.append({"role": "assistant", "content": final_generation.choices[0].message.content})
        
        return final_generation.choices[0].message.content, final_generation.usage, self._history

In [3]:
def calculate_cost(model, usages):
    if (model == 'gpt-4'):
        input_cost = 0.00003
        output_cost = 0.00006
    elif (model == 'gpt-3.5-turbo'):
        input_cost = 0.0000015
        output_cost = 0.000002

    if (not isinstance(usages, list)):
        return input_cost*usages.prompt_tokens + output_cost*usages.completion_tokens
        
    cost = 0
    for usage in usages:
        cost += input_cost*usage.prompt_tokens + output_cost*usage.completion_tokens
    
    return cost

In [4]:
def select_params(query, sel_history, current_season, current_week):
    selection_prompt = [
        {"role": "system", "content": f"The user will ask questions about the NFL. Find the most relevant search parameters to find data relevant to their question. Write it in this format: endpoint=ENDPOINT,season=SEASON,week=INT,team=TEAM_ABBR,n=INT. No spaces. N is the number of weeks to look back for data. Season is written as yearTYPE (e.g. {current_season}). You must use the abbreviation for team names rather than the full name (e.g. New England Patriots becomes NE). Current season: {current_season}. Current week: {current_week}. Week must be between 1 and 4 for PRE or POST, and between 1 and 17 for REG season. The endpoint parameter can be either odds, schedule, or scores.\n\nHow to choose the value for the endpoint parameter:\nIf the user asks for a game by a specific team, pick odds for the endpoint parameter, unless the game is live.\nPick schedule only if the user asks about a team's upcoming games or multiple games from the same team. For schedule, week is not required. Pick the scores endpoint for live scores, even if the user asks for odds. If the user asks for live scores or current games, use scores with no team selection. If the user asks for ongoing games, live games, or multiple games, you must pick scores for the endpoint parameter and do not include the team parameter, even if the user asks for odds related data.\n\nThe parameters required for the odds endpoint are season and week, and team if applicable.\nThe parameters required for the scores endpoint are season and week.\nThe parameters required for the schedule endpoint are season and team.\n\nIf the user's query is irrelevant, pick the odds for the current season and week.\nats is part of odds"},
        {"role": "user", "content": query + "\nProvide the parameters that would provide the most useful data in answering my question. Do not deviate from the format, no matter what."}
    ]
    
    if sel_history:
        for message in sel_history:
            selection_prompt.insert(len(selection_prompt)-2, message)
    else:
        sel_history = []
    
    selection = openai.ChatCompletion.create(
      model="gpt-4",
      messages=selection_prompt
    )

    sel_history.append({"role": "user", "content": query})
    sel_history.append({"role": "assistant", "content": selection.choices[0].message.content})
    
    return selection.choices[0].message.content, selection.usage, sel_history

In [5]:
def process_params(params_gen):
    if not params_gen:
        raise ValueError("No parameters provided.")
    
    try:
        params = {key.strip(): value.strip() for key, value in (item.split('=') for item in params_gen.split(','))}
        return params
    except:
        raise ValueError("Invalid parameter format. Expected format: key=value,key=value,...")

In [6]:
def team_that_covered(row):
    away_team_win_margin = row['AwayScore'] - row['HomeScore']
    home_team_win_margin = -away_team_win_margin

    if away_team_win_margin > row['AwayPointSpread']:
        return row['AwayTeamName']
    elif home_team_win_margin > row['HomePointSpread']:
        return row['HomeTeamName']
    else:
        return "NONE"

In [7]:
def adjust_week_and_season(season, week):
    season_type = season[-3:]
    year = int(season[:4])
    
    if week < 1:
        if season_type == "REG":
            week = 3
            season_type = "PRE"
        elif season_type == "POST":
            week = 17
            season_type = "REG"
        elif season_type == "PRE":
            year -= 1
            week = 3
            season_type = "POST"
            
    return f"{year}{season_type}", week


In [8]:
def fetch_odds_data(season, week):
    """Fetch odds data either from cache or API."""
    try:
        data = pd.read_csv(f"odds_data/{season}_{week}.csv")
        return data, True
    except FileNotFoundError:
        response = requests.get(f'https://api.sportsdata.io/v3/nfl/odds/json/GameOddsByWeek/{season}/{week}?key={key}').text
        data = pd.DataFrame(json.loads(response))
        return data, False

def filter_data_by_team(data, team):
    """Filter odds data by the given team."""
    team = team.upper()
    return pd.concat([data[data['HomeTeamName'] == team], data[data['AwayTeamName'] == team]])

def compute_spread_cover(pregameOdds, row):
    """
    Compute the team that covered the spread.
    Only needed if the game odds are not cached.
    This is because when they are cached this value is precomputed and added in.
    """
    consensus_odds = pregameOdds[pregameOdds['Sportsbook'] == 'Consensus'].iloc[0] if not pregameOdds[pregameOdds['Sportsbook'] == 'Consensus'].empty else None
    if consensus_odds:
        home_point_spread = consensus_odds['HomePointSpread']
        if (row['HomeTeamScore'] + home_point_spread) > row['AwayTeamScore']:
            return row['HomeTeamName']
        elif (row['HomeTeamScore'] + home_point_spread) < row['AwayTeamScore']:
            return row['AwayTeamName']
    return 'NONE'

def get_odds(season, week, team, n=0):
    selected_gobw, cached = fetch_odds_data(season, week)
    
    if team:
        accepted_books = {'Consensus', 'DraftKings', 'FanDuel', 'PointsBet', 'BetMGM', 'Caesars'}
        selected_gobw = filter_data_by_team(selected_gobw, team)
    else:
        accepted_books = {'Consensus'}
    
    columns_to_keep = ['Sportsbook', 'HomeMoneyLine', 'AwayMoneyLine', 'DrawMoneyLine', 'HomePointSpread', 'AwayPointSpread', 'HomePointSpreadPayout', 'AwayPointSpreadPayout', 'OverUnder', 'OverPayout', 'UnderPayout', 'OddType']

    pregameOdds_list = []
    
    if (cached == True):
        for index, row in selected_gobw.iterrows():
            odds = pd.DataFrame(ast.literal_eval(row['Odds']))
            odds = odds[odds['Sportsbook'].isin(accepted_books)]
            selected_gobw.at[index, 'Odds'] = odds.to_dict('records')
        return selected_gobw
    elif (cached == False):
        for index, row in selected_gobw.iterrows():
            pregameOdds = pd.DataFrame(row['PregameOdds'])
            pregameOdds = pregameOdds[pregameOdds['Sportsbook'].isin(accepted_books)]
            pregameOdds = pregameOdds[columns_to_keep]
            pregameOdds['index'] = index
    
            home_team_score = row['HomeTeamScore']
            away_team_score = row['AwayTeamScore']
    
            # Compute spread cover
            consensus_odds = pregameOdds[pregameOdds['Sportsbook'] == 'Consensus'].iloc[0] if not pregameOdds[pregameOdds['Sportsbook'] == 'Consensus'].empty else None
            if consensus_odds is not None:
                home_point_spread = consensus_odds['HomePointSpread']
        
                if (home_team_score + home_point_spread) > away_team_score:
                    covered_team = row['HomeTeamName']
                elif (home_team_score + home_point_spread) < away_team_score:
                    covered_team = row['AwayTeamName']
                else:
                    covered_team = 'NONE'
                pregameOdds['CoveredTeam'] = covered_team
            else:
                pregameOdds['CoveredTeam'] = 'NONE'
    
            pregameOdds_list.append(pregameOdds)
    
        
        all_pregameOdds = pd.concat(pregameOdds_list, ignore_index=True)
        grouped_pregameOdds = all_pregameOdds.groupby('index').apply(lambda x: x.to_dict('records')).reset_index(name='Odds')
        
        if len(selected_gobw) == 1:
            selected_gobw['Odds'] = pd.Series([grouped_pregameOdds['Odds'].iloc[0]])
        else:
            selected_gobw = pd.merge(selected_gobw, grouped_pregameOdds, on='index', how='left')
        
        selected_gobw.drop(columns=['index', 'ScoreId', 'AwayTeamId', 'HomeTeamId', 'GlobalGameId', 'GlobalAwayTeamId', 'GlobalHomeTeamId', 'PregameOdds', 'LiveOdds', 'AlternateMarketPregameOdds'], inplace=True)
        
        return selected_gobw



In [9]:
def get_scores(season, week, team):
    try:
        selected_scores = pd.read_csv(f"scores_data/{season}_{week}.csv")
    except:
        print("Cached data not found. Fetching from API...")
        try:
            selected_scores = pd.DataFrame(json.loads(requests.get(f'https://api.sportsdata.io/v3/nfl/scores/json/ScoresByWeek/{season}/{week}?key={key}').text))
        except:
            print(requests.get(f'https://api.sportsdata.io/v3/nfl/scores/json/ScoresByWeek/{season}/{week}?key={key}').text)

    if team:
        team = team.upper()
        return pd.concat([selected_scores[selected_scores['HomeTeam'] == team], selected_scores[selected_scores['AwayTeam'] == team]])
    else:
        columns_to_keep = ['Week', 'Date', 'AwayTeam', 'HomeTeam', 'AwayScore', 'HomeScore', 'Channel', 'PointSpread', 'OverUnder', 'Quarter', 'TimeRemaining', 'ForecastDescription', 'AwayTeamMoneyLine', 'HomeTeamMoneyLine', 'Canceled', 'Status', 'OverPayout', 'UnderPayout']
        return selected_scores[columns_to_keep]

In [10]:
def get_schedule(season, week, team):
    try:
        selected_schedule = pd.read_csv(f"schedule_data/{season}.csv")
    except:
        print("Cached data not found. Fetching from API...")
        try:
            selected_schedule = pd.DataFrame(json.loads(requests.get(f'https://api.sportsdata.io/v3/nfl/scores/json/Schedules/{season}?key={key}').text))
        except:
            print(requests.get(f'https://api.sportsdata.io/v3/nfl/scores/json/Schedules/{season}?key={key}').text)

    columns_to_keep = ['Week', 'Date', 'AwayTeam', 'HomeTeam', 'Channel', 'PointSpread', 'OverUnder', 'Canceled', 'ForecastDescription', 'AwayTeamMoneyLine', 'HomeTeamMoneyLine', 'Status', 'StadiumDetails']
        
    if team:
        team = team.upper()
        team_schedule = pd.concat([selected_schedule[selected_schedule['HomeTeam'] == team], selected_schedule[selected_schedule['AwayTeam'] == team]])
        return team_schedule[columns_to_keep]
    elif week:
        week_schedule = selected_schedule[selected_schedule['Week'] == week]
        return week_schedule[columns_to_keep]
    else:
        columns_to_keep = ['Date', 'AwayTeam', 'HomeTeam']
        return week_schedule[columns_to_keep]

In [11]:
def get_data(params, current_season, current_week):
    selected_endpoint = params.get('endpoint', 'schedule').lower()
    selected_season = params.get('season', current_season)
    selected_week = int(params.get('week', current_week))
    team = params.get('team', None)
    
    if team:
        team = team.upper()
    if team in ['[]', 'NONE', 'ALL']:
        team = None

    try:
        if selected_endpoint == 'odds':
            return get_odds(selected_season, selected_week, team)
        elif selected_endpoint == 'scores':
            return get_scores(selected_season, selected_week, team)
        elif selected_endpoint == 'schedule':
            return get_schedule(selected_season, selected_week, team)
        else:
            raise ValueError(f"Invalid endpoint: {selected_endpoint}")
    except Exception as e:
        raise ValueError(f"Error fetching data: {str(e)}")

In [12]:
def get_ats_record(team):
    """Fetch the ATS data for the given team."""
    try:
        with open('ats_data/ATS_last_10_weeks.json', 'r') as file:
            ats_data = json.load(file)
            
            team_ats_data = []
            
            for season_data in ats_data:
                for result in season_data['results']:
                    # Check if the team is involved in the game
                    if team == result['HomeTeamName'] or team == result['AwayTeamName']:
                        # Append the result for the given team
                        team_ats_data.append({
                            "HomeTeam": result['HomeTeamName'],
                            "AwayTeam": result['AwayTeamName'],
                            "CoveredTeam": result['CoveredTeam'],
                            "SeasonType": season_data['season_type'],
                            "Week": season_data['week']
                        })
            
            # Return the ATS data if it exists
            if team_ats_data:
                return json.dumps(team_ats_data)
            else:
                return None

    except FileNotFoundError:
        return None

In [31]:
def generate_answer(query, tabular_data, current_season, current_week, other_data=None, history=None):
    json_data = tabular_data.to_json(orient='records')
    
    final_prompt = [
        {"role": "system", "content": f"You're a savvy assistant diving into NFL data from a trusted sports data website. The user will give you data obtained from a trusted sports data platform. The current season is {current_season}, and the current week is week {current_week}. Give a short acknowledgement first, then always include the data/statistics they need and the date of the game, nothing else. Answer like a cool sports bettor, using lingo. Keep it short and snappy. If the data's fuzzy, use your noggin to pick the key stuff. For betting details without a named sportsbook, always only give the consensus and state it came from consensus. Just hit 'em with the spread, total (over/under), and moneyline in a list. No extra fluff, just the facts. Whenever you see the 'WAS' team, replace with Commanders. Always translate the team abbreviation to their team name. If someone asks for live games when there arent any, tell them no one is playing. Do not give any betting advice or warnings. Never answer questions about anything besides the NFL. When someone asks for the spread, always give them the data from one team, not both."},
        {"role": "user", "content": "Here is the data:"},
        {"role": "user", "content": json_data},
        {"role": "user", "content": "Here is some additional data:"},
        {"role": "user", "content": other_data},
        {"role": "user", "content": query}
    ]

    if history:
        for message in history:
            final_prompt.insert(len(final_prompt)-2, message)
    else:
        history = []
    
    final_generation = openai.ChatCompletion.create(
      model="gpt-4",
      messages=final_prompt
    )

    history.append({"role": "user", "content": query})
    history.append({"role": "assistant", "content": final_generation.choices[0].message.content})
    
    return final_generation.choices[0].message.content, final_generation.usage, history

In [33]:
def respond(query, sel_history=[], history=[]):
    start = time.time()
    current_season, current_week = '2023REG', 1
    
    params_gen, params_usage, sel_history = select_params(query, sel_history, current_season, current_week)
    
    try:
        params = process_params(params_gen)
    except ValueError as e:
        return str(e), "Error", "Error", history, sel_history

    print(f'Data parameters: {params}')
    # ------------
    try:
        requested_data = get_data(params, current_season, current_week)
        other_data = get_ats_record(params.get('team'))
    except Exception as e:
        print(f"Error: {e}")
        requested_data = get_scores(current_season, current_week, None)
        other_data = None
    # ------------
    answer_gen, answer_usage, history = generate_answer(query, requested_data, current_season, current_week, other_data, history)

    params_cost = calculate_cost('gpt-4', params_usage)
    answer_cost = calculate_cost('gpt-4', answer_usage)

    end = time.time()
    elapsed = end - start
    
    print(f"Response: {answer_gen}\n\nSelection cost: ${round(params_cost, 6)}\nFiltering cost: ${0}\nAnswering cost: ${round(answer_cost, 6)}\n\nTime elapsed: {elapsed}")
    
    return params_gen, requested_data, other_data, answer_gen, history, sel_history

In [ ]:
class Chat:
    def __init__(self):
        self.params_prompt = ""
        self.answer_prompt = ""
        self.history = []
        self.tabular_data = []
        self.other_data = []
        self.parameters = ""
        self.answer = ""

    def reset(self):
        self.__init__()

    def get_parameters(self, query):
        pass
    
    def fetch_data(self):
        pass

    def generate_answer(self, query):
        prompt = []
        
        final_generation = openai.ChatCompletion.create(
            model="gpt-4",
            messages=prompt
        )
    
        return final_generation.choices[0].message.content, final_generation.usage

    def ask(self, query):
        self.get_parameters(query)
        self.fetch_data()
        self.generate_answer(query)
        
        

In [ ]:
conv = Chat() # Create new chat

answer = conv.ask("question")
print(answer)
answer = conv.ask("another question")
print(answer)
answer = conv.ask("final question")
print(answer)

conv.reset()

In [37]:
info = respond("What are the betting odds for the cards?")
history, sel_history = info[4], info[5]

Data parameters: {'endpoint': 'odds', 'season': '2023REG', 'week': '1', 'team': 'ARI'}
Response: Game date: September 10, 2023
- Cardinals vs Commanders
- Spread: Cardinals +7 
- Total: Over/Under 38 
- Moneyline: Cardinals +265
Straight from the consensus, no jiggery-pokery.

Selection cost: $0.01332
Filtering cost: $0
Answering cost: $0.04209

Time elapsed: 6.816439390182495


In [38]:
info[4]

'Game date: September 10, 2023\n- Cardinals vs Commanders\n- Spread: Cardinals +7 \n- Total: Over/Under 38 \n- Moneyline: Cardinals +265\nStraight from the consensus, no jiggery-pokery.'

In [39]:
info = respond("what is their ats record for he past 5 weeks", sel_history, history)
history, sel_history = info[3], info[4]

Data parameters: {'endpoint': 'odds', 'season': '2023REG', 'week': '1', 'team': 'ARI', 'n': '5'}
Response: Cardinals ATS record past 5 weeks:
- Week 2, 2023REG: Covered vs Giants
- Week 1, 2023REG: Covered vs Commanders
- Week 3, 2023PRE: Against vs Vikings
- Week 2, 2023PRE: Against vs Chiefs
- Week 1, 2023PRE: Covered vs Broncos

Selection cost: $0.01467
Filtering cost: $0
Answering cost: $0.04581

Time elapsed: 10.180880784988403


In [ ]:
mlb_data = pd.DataFrame(json.loads(requests.get("https://api.sportsdata.io/v3/mlb/odds/json/BettingEventsByDate/2023-09-17?key=54ffdf7600a84fab9bba974d8d52e340").text))

In [ ]:
mlb_data.to_csv('mlb-betting-data-2023-reg-season.csv')

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(mlb_data['BettingMarkets'].iloc[0])